In [14]:
import pandas as pd
import sys
import os
import synapseclient
import itertools
import numpy as np
import re

#local libs
sys.path.append("/home/apratap/dev/appys/lib/")
import synapseutils
import utils

syn = synapseclient.login()

Welcome, Abhishek Pratap!


### Source the functions for processing OHSU plates

In [15]:
#plate maps
%run '/home/apratap/dev/AML/DrugScreenHarmonization/OHSU_plates/OHSU_plateMaps.py'

#plate reader functions
%run '~/dev/AML/DrugScreenHarmonization/OHSU_plates/common_plate_reader_functions.ipynb'

Welcome, Abhishek Pratap!


### Get data for all OHSU Samples done at Phoenix 

In [16]:
dataFiles_df = syn.query("select id from entity where parentId == 'syn4932092'")
dataFiles_df = synapseutils.query2df(dataFiles_df)
dataFiles = [ syn.get(x).path for x in dataFiles_df.id ]

### Process ENV plate reader data

In [17]:
dataFiles_ENV_reader = [ x for x in dataFiles if x.find('Env') != -1]
mdata_ENV_reader = [ os.path.basename(x).replace('.xlsx','').split('_') for x in dataFiles_ENV_reader ]
mdata_ENV_reader = pd.DataFrame.from_records(mdata_ENV_reader, columns=['run_date', 'disease', 'set', 'assay', 'read_time',  'plate_reader'])
mdata_ENV_reader['plate_origin'] = 'OHSU'
mdata_ENV_reader['run_at'] = 'Phoenix'
mdata_ENV_reader['plate_origin'] = 'OHSU'
mdata_ENV_reader['file'] = dataFiles_ENV_reader
mdata_ENV_reader['plate_version'] = 'MarcTest123_v6'
mdata_ENV_reader.drop('disease', axis=1, inplace=True)

In [18]:
data_ENV_reader = list()
for num,row in mdata_ENV_reader.iterrows():
    temp_data = process_ENV1_plate_reader(row.file)    
    #add the metadata to processed raw data
    for key,val in row.iterkv():
        temp_data[key]=val
    data_ENV_reader.append(temp_data)
#final pandas df
data_ENV_reader  = pd.concat(data_ENV_reader, axis=0)

### Process PAR plate reader data

In [19]:
dataFiles_PAR_reader = [ x for x in dataFiles if x.find('Env') == -1]
mdata_PAR_reader = [ os.path.basename(x).replace('_All_Data.xlsx','').split('_') for x in dataFiles_PAR_reader ]
mdata_PAR_reader = pd.DataFrame.from_records(mdata_PAR_reader, columns=['run_date', 'plate_origin', 'set', 'assay', 'plate_reader'])
mdata_PAR_reader['plate_origin'] = 'OHSU'
mdata_PAR_reader['run_at'] = 'Phoenix'
mdata_PAR_reader['plate_origin'] = 'OHSU'
mdata_PAR_reader['file'] = dataFiles_PAR_reader
mdata_PAR_reader['plate_version'] = 'MarcTest123_v6'

In [20]:
data_PAR_reader = list()
import xlrd
#custom code to read 4 sheets in each excel file for PAR plate reader
#each sheet corresponds to a specific read time
for num,row in mdata_PAR_reader.iterrows():
    xls = xlrd.open_workbook(row.file)
    for sheet in xls.sheet_names():
        temp_mdata = sheet.split('_')
        read_time = temp_mdata[5]
        temp_data = process_PAR1_plate_reader(row.file, sheetname=sheet)
        #add the metadata to processed raw data
        temp_data['read_time'] = read_time
        for key,val in row.iterkv():
            temp_data[key]=val
        data_PAR_reader.append(temp_data)

#final pandas df
data_PAR_reader  = pd.concat(data_PAR_reader, axis=0)
    

In [21]:
final_data = pd.concat([data_ENV_reader, data_PAR_reader], axis=0)

### Join with the plate map to get the meta data

In [22]:
final_data = pd.merge(OHSU_v6_plateMap,final_data, how='right', 
                            left_on = ['plate_version','Plate_Num', 'Well_Row', 'Well_Column'],
                            right_on = ['plate_version', 'plateNum' ,'row', 'col'])

### Normalize the plate data

In [23]:
def calc_norm_factors(df):
    
    #taking mean of positive control on this plate
    pos_control = df.query('row == "H" & (col == 8|col == 16)')['value'].mean()
    
    #ANYTHING EXCEPT ROW H FOR COL 8 AND 16 AND ALL ROWS FOR COL 24
    median_DMSO_col8 = df.query('row != "H" & col == 8')['value'].tolist()
    median_DMSO_col16 = df.query('row != "H" & col == 16')['value'].tolist()
    median_DMSO_col24 = df.query('col == 24')['value'].tolist()
    median_DMSO = np.median(median_DMSO_col8 + median_DMSO_col16 + median_DMSO_col24)
    return pd.DataFrame({'pos_control': [pos_control], 'median_DMSO': [median_DMSO]})

grp = final_data.groupby(['plate_version', 'plate_origin' ,'run_at','assay', 'plateNum' , 
                          'plate_reader' ,'read_time', 'set'])
norm_factors = grp.apply(calc_norm_factors).reset_index()

#merge the normFactors
final_data = pd.merge(final_data, norm_factors)

#normValue
final_data['normValue'] = (final_data.value - final_data.pos_control) / ( final_data.median_DMSO - final_data.pos_control)
final_data.normValue = final_data.normValue.map(lambda x: np.around(x, 4))

In [35]:
required_columns = ['plate_version', 'plateNum','plate_origin' , 'run_at'  ,'run_date', 'set' ,'assay' ,
                    'plate_reader', 'read_time' , 'row' ,'col', 'Drug', 'drug_replicate_across_plates', 'Concentration', 'value', 'Unit', 
                    'normValue' , 'pos_control' ,'median_DMSO']
final_data = final_data[required_columns]

### Upload to Synapse

In [ ]:
outfile = 'patient_sample_OHSU_plates_run_at_Phoenix_rawData.tsv'
final_data.to_csv(outfile, sep = '\t', index=False)
syn.store(synapseclient.File(outfile, parentId = 'syn4932396'),
          used = dataFiles_df.id.tolist(),
          executed ='')
!rm $outfile

In [36]:
dataFiles_df

,id
0,syn4932102
1,syn4932106
2,syn4932103
3,syn4932105
4,syn4932110
5,syn4932114
6,syn4932112
7,syn4932113
8,syn4932121
9,syn4932122
